In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import sys
import os
sys.path.append("../")
import matplotlib.pyplot as plt
import time

In [ ]:
import preprocess_blockSVD as pre_svd
import trefide as tfd
import spatial_filtering as sp_filters
#from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
#import scipy as sp
import tools as tools
import cvxpy as cp
import caiman as cm


In [ ]:
import h5py

project_dir = '/data/lab/Dropbox/QState_IntrinsicExcitability/'
fname = 'IntrinsicExcitability_movie017.mat'
#data = sio.loadmat(project_dir+fname)
with h5py.File(os.path.join(project_dir, fname)) as f:
    m_orig = f['mov']['data'].value.T
    sampleRate = f['mov']['sampleRate'].value
    sampleRate_units = f['mov']['sampleRate_units'].value
    print(list(f['mov'].keys()))

In [ ]:
print(m_orig.shape)
m_orig_ = m_orig[5:-5,150:220,2100:3000].transpose([2,0,1])
print(m_orig_.shape)
plt.imshow(m_orig_[0,:,:])

In [ ]:
def remove_trend(Y_rm,detrend_option='linear'):
    mean_pixel = Y_rm.mean(axis=1,keepdims=True)
    Y_rm2 = Y_rm - mean_pixel
    # Detrend linear 
    if detrend_option=='linear':
        detr_data = sp.signal.detrend(Y_rm2[:,:],axis=1,type='l')
    elif detrend_option=='quad':
        detr_data = detrend(Y_rm)
    Y_det = detr_data + mean_pixel
    offset = Y_rm - Y_det
    return Y_det, offset

In [ ]:
fig = plt.figure(figsize=(10,5))
plt.plot(m_orig_[:,0,0])

In [ ]:
Frame = cm.movie(m_orig_)#[100:5100,1:-5,28:]
Frame.play(gain=5, fr=10, magnification=5)

In [ ]:
images_ = m_orig_.transpose([1,2,0])
dims = images_.shape
print(dims)

In [ ]:
# Caserati representation d x T
images_r = images_.reshape((np.prod(dims[:2]),dims[2]),order='F')

# if temporal downsampling
if False:
    ts = 5
    images_r = pre_svd.temporal_decimation(images_r.T,ts).T
    dims = dims[0], dims[1], dims[2]//ts
# get rid of trend
if True:
    images_detr, offset = tools.remove_trend(images_r)
    images_r = images_detr.copy()
else:
    offset = np.zeros(images_r.shape)

In [ ]:
# Run wiener filter
if True:
    # run WF on Y
    images_wf,_ = sp_filters.spatial_filter_image(images_r.reshape(dims,order='F'))
    # cm.movie(images_wf).transpose([2,0,1]).play(magnification=10)
    images_wf = images_wf.reshape((np.prod(dims[:2]),dims[2]),order='F')
else:
    images_wf = images_r.copy()

In [ ]:
plt.imshow(images_wf.reshape(dims,order='F')[:,:,0])
plt.colorbar()

In [ ]:
# subtract off the mean,
mean_px = images_wf.mean(1,keepdims=True)
images_wf = images_wf - mean_px
if True:
    # compute noise level
    # and normalize each pixel to have unit noise level
    noise_level = sp_filters.noise_estimator(images_wf[:,:],range_ff=[0.25,0.5])
    image_= images_wf/noise_level[:,np.newaxis]
    # confirm noise variance =1 in each pixel
    tmp_ = sp_filters.noise_estimator(image_[:,:],range_ff=[0.25,0.5])
    print(tmp_.min())
    print(tmp_.max())
else:
    image_ = images_wf

In [ ]:
plt.imshow(noise_level.reshape(dims[:2],order='F'))
plt.colorbar()
plt.show()
plt.imshow(tmp_.reshape(dims[:2],order='F')-1)
plt.colorbar()

In [ ]:
# extract background (WF and 1p data)
if False:
    U,s,Vt = pre_svd.compute_svd(image_,method='randomized',n_components=2)
    movie_rank2 = U.dot(np.diag(s).dot(Vt))
    residual = image_ - movie_rank2
    W = residual.reshape(dims,order='F')
else:
    W = image_.reshape(dims,order='F')

In [ ]:
k = 25

In [ ]:
maxlag = 5#5
confidence = 0.95
min_rank = 1
Yd1,_ = pre_svd.svd_patch(W,k=k,maxlag=maxlag,confidence=confidence,
                                greedy=False,fudge_factor=1,
                                 mean_th_factor=2.,min_rank=min_rank)

In [ ]:
# greedy for a single tile (~ neuron size)
patches = pre_svd.split_image_into_blocks(W,k)
start= time.time()
b,_ = pre_svd.compress_dblocks(patches[1],
                                maxlag=maxlag,confidence=confidence,
                                greedy=True,fudge_factor=0.9,
                                mean_th_factor=2,U_update=True,
                              verbose=True,plot_en=True,min_rank=1)
print('Run for %.f'%(time.time()-start))


In [ ]:
Yd,_ = pre_svd.svd_patch(W,k=k,maxlag=maxlag,confidence=confidence,
                         greedy=True,fudge_factor=0.9,mean_th_factor=2,min_rank=1,
                         U_update=True,plot_en=False)

In [ ]:
def redo_(W_,dims,noise_level,mean_px):
    W = W_.reshape((np.prod(dims[:2]),dims[2]),order='F').copy()
    W= W*noise_level[:,np.newaxis]
    W+= mean_px
    W = W.reshape(dims,order='F')
    return W

In [ ]:
# reconstruct matrix from Yd
images_rd_= images_r.reshape(dims,order='F')
Yd1_ = redo_(Yd1,dims,noise_level,mean_px) #vanilla
Yd_ = redo_(Yd,dims,noise_level,mean_px) #greedy

In [ ]:
M1 = cm.movie(np.asarray(images_rd_).transpose([2,0,1]))
M2 = cm.movie(np.asarray(Yd1_).transpose([2,0,1]))
M3 = cm.movie(np.asarray(Yd_).transpose([2,0,1]))

M4  = M1 - M2
M5  = M1 - M3

MZ = np.zeros(M1.shape)
if False:
    M1 = M1-M1.min(0,keepdims=True)
    M2 = M2-M2.min(0,keepdims=True)
    M3 = M3-M3.min(0,keepdims=True)
    M4 = M4-M4.min(0,keepdims=True)
    M5 = M5-M5.min(0,keepdims=True)

cm.concatenate([
    cm.concatenate([M1,M2,M3],axis=2),
    cm.concatenate([MZ,M4,M5],axis=2)
    ],axis=1).play(gain=1,magnification=4)